# Stitch 2 pointclouds together

In [ ]:
import open3d as o3d
import teaserpp_python
import numpy as np 
import copy
from helpers import *

VOXEL_SIZE = 0.05
VISUALIZE = True

# Load and visualize two point clouds from 3DMatch dataset
A_pcd_raw = o3d.io.read_point_cloud('/home/szz/Documents/localizer_blueprint/examples/teaser_python_fpfh_icp/data/office_test.pcd')
B_pcd_raw = o3d.io.read_point_cloud('/home/szz/Downloads/test.pcd')
A_pcd_raw.paint_uniform_color([0.0, 0.0, 1.0]) # show A_pcd in blue
B_pcd_raw.paint_uniform_color([1.0, 0.0, 0.0]) # show B_pcd in red
if VISUALIZE:
    o3d.visualization.draw_geometries([A_pcd_raw,B_pcd_raw]) # plot A and B 

# voxel downsample both clouds
A_pcd = A_pcd_raw.voxel_down_sample(voxel_size=VOXEL_SIZE)
B_pcd = B_pcd_raw.voxel_down_sample(voxel_size=VOXEL_SIZE)
if VISUALIZE:
    o3d.visualization.draw_geometries([A_pcd,B_pcd]) # plot downsampled A and B 

A_xyz = pcd2xyz(A_pcd) # np array of size 3 by N
B_xyz = pcd2xyz(B_pcd) # np array of size 3 by M

# extract FPFH features
A_feats = extract_fpfh(A_pcd,VOXEL_SIZE)
B_feats = extract_fpfh(B_pcd,VOXEL_SIZE)

# establish correspondences by nearest neighbour search in feature space
corrs_A, corrs_B = find_correspondences(
    A_feats, B_feats, mutual_filter=True)
A_corr = A_xyz[:,corrs_A] # np array of size 3 by num_corrs
B_corr = B_xyz[:,corrs_B] # np array of size 3 by num_corrs

num_corrs = A_corr.shape[1]
print(f'FPFH generates {num_corrs} putative correspondences.')

# visualize the point clouds together with feature correspondences
points = np.concatenate((A_corr.T,B_corr.T),axis=0)
lines = []
for i in range(num_corrs):
    lines.append([i,i+num_corrs])
colors = [[0, 1, 0] for i in range(len(lines))] # lines are shown in green
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([A_pcd,B_pcd,line_set])

# robust global registration using TEASER++
NOISE_BOUND = VOXEL_SIZE
teaser_solver = get_teaser_solver(NOISE_BOUND)
teaser_solver.solve(A_corr,B_corr)
solution = teaser_solver.getSolution()
R_teaser = solution.rotation
t_teaser = solution.translation
T_teaser = Rt2T(R_teaser,t_teaser)

# Visualize the registration results
A_pcd_T_teaser = copy.deepcopy(A_pcd).transform(T_teaser)
o3d.visualization.draw_geometries([A_pcd_T_teaser,B_pcd])

# local refinement using ICP
icp_sol = o3d.pipelines.registration.registration_icp(
      A_pcd, B_pcd, NOISE_BOUND, T_teaser,
      o3d.pipelines.registration.TransformationEstimationPointToPoint(),
      o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100))
T_icp = icp_sol.transformation

print (T_icp)

# visualize the registration after ICP refinement
A_pcd_T_icp = copy.deepcopy(A_pcd).transform(T_icp)
o3d.visualization.draw_geometries([A_pcd_T_icp,B_pcd])

# Stitch multiple pointclouds together

In [ ]:
import open3d as o3d
import teaserpp_python
import numpy as np
import copy
from helpers import *

VOXEL_SIZE = 0.02
VISUALIZE = True

def process_point_clouds(A_pcd_raw, B_pcd_raw, voxel_size):
 try:
 # voxel downsample both clouds
 A_pcd = A_pcd_raw.voxel_down_sample(voxel_size=voxel_size)
 B_pcd = B_pcd_raw.voxel_down_sample(voxel_size=voxel_size)
 if VISUALIZE:
 o3d.visualization.draw_geometries([A_pcd, B_pcd]) # plot downsampled A and B

 A_xyz = pcd2xyz(A_pcd) # np array of size 3 by N
 B_xyz = pcd2xyz(B_pcd) # np array of size 3 by M

 # extract FPFH features
 A_feats = extract_fpfh(A_pcd, voxel_size)
 B_feats = extract_fpfh(B_pcd, voxel_size)

 # establish correspondences by nearest neighbour search in feature space
 corrs_A, corrs_B = find_correspondences(A_feats, B_feats, mutual_filter=True)
 A_corr = A_xyz[:, corrs_A] # np array of size 3 by num_corrs
 B_corr = B_xyz[:, corrs_B] # np array of size 3 by num_corrs

 num_corrs = A_corr.shape[1]
 print(f'FPFH generates {num_corrs} putative correspondences.')

 # visualize the point clouds together with feature correspondences
 points = np.concatenate((A_corr.T, B_corr.T), axis=0)
 lines = [[i, i + num_corrs] for i in range(num_corrs)]
 colors = [[0, 1, 0] for _ in range(len(lines))] # lines are shown in green
 line_set = o3d.geometry.LineSet(
 points=o3d.utility.Vector3dVector(points),
 lines=o3d.utility.Vector2iVector(lines),
 )
 line_set.colors = o3d.utility.Vector3dVector(colors)
 o3d.visualization.draw_geometries([A_pcd, B_pcd, line_set])

 # robust global registration using TEASER++
 NOISE_BOUND = voxel_size
 teaser_solver = get_teaser_solver(NOISE_BOUND)
 teaser_solver.solve(A_corr, B_corr)
 solution = teaser_solver.getSolution()
 R_teaser = solution.rotation
 t_teaser = solution.translation
 T_teaser = Rt2T(R_teaser, t_teaser)

 # Visualize the registration results
 A_pcd_T_teaser = copy.deepcopy(A_pcd).transform(T_teaser)
 o3d.visualization.draw_geometries([A_pcd_T_teaser, B_pcd])

 # local refinement using ICP
 icp_sol = o3d.pipelines.registration.registration_icp(
 A_pcd, B_pcd, NOISE_BOUND, T_teaser,
 o3d.pipelines.registration.TransformationEstimationPointToPoint(),
 o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100))
 T_icp = icp_sol.transformation

 # visualize the registration after ICP refinement
 A_pcd_T_icp = copy.deepcopy(A_pcd).transform(T_icp)
 o3d.visualization.draw_geometries([A_pcd_T_icp, B_pcd])

 # Combine the two point clouds into one
 A_points = np.asarray(A_pcd_T_icp.points)
 B_points = np.asarray(B_pcd.points)

 # Merge the points and colors (if any)
 merged_points = np.vstack((A_points, B_points))

 # Create a new point cloud with the merged points
 merged_pcd = o3d.geometry.PointCloud()
 merged_pcd.points = o3d.utility.Vector3dVector(merged_points)

 # Optionally, you can also merge the colors if you want to preserve them
 if A_pcd_T_icp.has_colors() and B_pcd.has_colors():
 A_colors = np.asarray(A_pcd_T_icp.colors)
 B_colors = np.asarray(B_pcd.colors)
 merged_colors = np.vstack((A_colors, B_colors))
 merged_pcd.colors = o3d.utility.Vector3dVector(merged_colors)

 return merged_pcd
 except Exception as e:
 print(f"Error processing point clouds: {e}")
 return None

def read_point_cloud(file_or_pcd):
 if isinstance(file_or_pcd, str):
 return o3d.io.read_point_cloud(file_or_pcd)
 elif isinstance(file_or_pcd, o3d.geometry.PointCloud):
 return file_or_pcd
 else:
 raise TypeError("Input must be a file path or a PointCloud object")

def merge_pairwise(pcd_list, voxel_size):
 # Merge point clouds in pairs
 merged_pcd_list = []
 for i in range(0, len(pcd_list), 2):
 if i + 1 < len(pcd_list):
 pcd1 = read_point_cloud(pcd_list[i])
 pcd2 = read_point_cloud(pcd_list[i + 1])
 merged_pcd = process_point_clouds(pcd1, pcd2, voxel_size)
 merged_pcd_list.append(merged_pcd)
 else:
 # If there is an odd number of point clouds, add the last one directly
 pcd = read_point_cloud(pcd_list[i])
 merged_pcd_list.append(pcd)
 return merged_pcd_list

def merge_multiple_point_clouds(pcd_paths, voxel_size):
 while len(pcd_paths) > 1:
 pcd_paths = merge_pairwise(pcd_paths, voxel_size)
 return pcd_paths[0] if pcd_paths else None

# List of point cloud files to merge
pcd_files = ['back_side_front.pcd','back_side_left.pcd', 'helmet_top.ply', 'back_side_right.pcd']

# Merge all point clouds
final_merged_pcd = merge_multiple_point_clouds(pcd_files, VOXEL_SIZE)

# Save the final merged point cloud
if final_merged_pcd:
 o3d.io.write_point_cloud("final_merged.ply", final_merged_pcd)
else:
 print("Merging failed.")